# Import packages


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    explained_variance_score,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)
from sktime.transformations.series.summarize import WindowSummarizer
from sktime.utils.plotting import plot_series

pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", None)

# Load data


In [ ]:
df = pd.read_parquet("../data/aggregated_data.parquet")

In [ ]:
df.head()

Keep only columns with high correlation with the drought column


In [ ]:
columns_to_keep = [
    "PRECTOT",
    "PS",
    "T2M_MAX",
    "T2M_RANGE",
    "WS10M_RANGE",
]

In [ ]:
X = df[columns_to_keep]
y = df[["drought"]]

In [ ]:
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X, test_size=0.3)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
print(f"Number of records in X_train: {X_train.shape}")
print(f"Number of records in X_test: {X_test.shape}")

In [ ]:
print(
    f"Range of dates in X_train: {X_train.index.get_level_values('date').min(), X_train.index.get_level_values('date').max()}"
)

print(
    f"Range of dates in X_test: {X_test.index.get_level_values('date').min(), X_test.index.get_level_values('date').max()}"
)

# Feature engineering


In [ ]:
Z_train = pd.concat([X_train, y_train], axis=1)
Z_test = pd.concat([X_test, y_test], axis=1)

In [ ]:
Z_train.head()

In [ ]:
kwargs = {
    "lag_feature": {
        "lag": [1, 2, 3, 4],
    }
}

target_cols = ["PRECTOT", "PS", "T2M_MAX", "T2M_RANGE", "WS10M_RANGE", "drought"]

transformer = WindowSummarizer(**kwargs, target_cols=target_cols)

In [ ]:
Z_train.head()

In [ ]:
Z_train_transformed = transformer.fit_transform(Z_train)

In [ ]:
Z_test_transformed = transformer.fit_transform(Z_test)

In [ ]:
Z_train_transformed.head()

In [ ]:
Z_test.head()

In [ ]:
Z_test_transformed.head()

In [ ]:
print(Z_train_transformed.columns)

# Utility functions


In [ ]:
def get_data_by_state(state_name: str):
    """
    Extracts and returns training and testing data for a specific state.
    Parameters:
    state_name (str): The name of the state for which the data is to be extracted.
    Returns:
    tuple: A tuple containing four DataFrames:
        - state_X_train: Training features for the specified state.
        - state_X_test: Testing features for the specified state.
        - state_y_train: Training labels for the specified state.
        - state_y_test: Testing labels for the specified state.
    """
    state_X_train = (
        Z_train_transformed[
            Z_train_transformed.index.get_level_values("state_name") == state_name
        ]
        .reset_index()
        .drop(columns="state_name")
    )

    state_X_test = (
        Z_test_transformed[
            Z_test_transformed.index.get_level_values("state_name") == state_name
        ]
        .reset_index()
        .drop(columns="state_name")
    )

    state_y_train = (
        y_train[y_train.index.get_level_values("state_name") == state_name]
        .reset_index()
        .drop(columns="state_name")
    )

    state_y_test = (
        y_test[y_test.index.get_level_values("state_name") == state_name]
        .reset_index()
        .drop(columns="state_name")
    )

    return (state_X_train, state_X_test, state_y_train, state_y_test)

In [ ]:
def plot_train_test_prediction_series(
    y_train: pd.DataFrame, y_test: pd.DataFrame, y_pred: np.array, state_name: str
):
    """
    Plots the training, testing, and predicted time series data for drought values and saves the plot as a PNG file.
    Parameters:
    y_train (pd.DataFrame): The training data series.
    y_test (pd.DataFrame): The testing data series.
    y_pred (np.array): The predicted data series.
    state_name (str): The name of the state for labeling the plot and saving the file.
    Returns:
    None
    """
    fig, ax = plot_series(
        y_train,
        y_test,
        pd.DataFrame(data=y_pred, index=y_test.index, columns=["drought"]),
        labels=[
            f"{state_name} y_train",
            f"{state_name} y_test",
            f"{state_name} y_pred",
        ],
    )

    # Format the x-axis to display only year and month
    # date_format = DateFormatter("%Y-%m")
    # ax.xaxis.set_major_formatter(date_format)
    ax.tick_params(axis="x", rotation=10)  # Adjust the rotation angle as needed
    ax.set_xlabel("Time")
    ax.set_ylabel("Drought value")

    plt.savefig(f"../figures/time-series-plots/{state_name}_time_series_plot.png")

    # plt.show()

# Modelling


In [ ]:
states_of_interest = [
    "Nevada",  # very high
    "Utah",  # very high
    "Nebraska",  # high
    "Texas",  # high
    "Mississippi",  # medium
    "Missouri",  # medium
    "New York",  # low
    "Ohio",  # low
]

In [ ]:
states_models = {}

for state in states_of_interest:
    state_X_train, state_X_test, state_y_train, state_y_test = get_data_by_state(state)

    state_model = LinearRegression()

    state_model.fit(
        state_X_train.drop(columns=["date"]).fillna(0.0),
        state_y_train.drop(columns=["date"]),
    )

    state_y_pred = state_model.predict(state_X_test.drop(columns=["date"]).fillna(0.0))

    states_models[state] = {
        "X_train": state_X_train,
        "X_test": state_X_test,
        "y_train": state_y_train,
        "y_test": state_y_test,
        "y_pred": state_y_pred,
        "model": state_model,
    }

In [ ]:
for state in states_of_interest:
    plot_train_test_prediction_series(
        states_models.get(state)["y_train"].set_index("date"),
        states_models.get(state)["y_test"].set_index("date"),
        states_models.get(state)["y_pred"],
        state,
    )

# Performance Analysis


In [ ]:
def residual_sum_of_errors(y_true, y_pred):
    residuals = y_true - y_pred
    rse = np.sum(residuals**2)
    return rse

In [ ]:
def root_mean_squared_error(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return rmse

In [ ]:
def evaluate(state_name):
    """
    Evaluate the performance of a regression model for a given state.
    Parameters:
    state_name (str): The name of the state for which the model evaluation is to be performed.
    Returns:
    dict: A dictionary containing the evaluation metrics for the model, including:
        - "State": The name of the state.
        - "Explained Variance": The explained variance score of the model.
        - "Mean Absolute Error": The mean absolute error of the model.
        - "Mean Squared Error": The mean squared error of the model.
        - "R^2 Score": The R^2 score of the model.
        - "Residual Sum of Errors": The residual sum of errors of the model.
        - "Root Mean Squared Error": The root mean squared error of the model.
    """
    y_true = states_models.get(state_name)["y_test"].drop(columns=["date"]).values
    y_pred = states_models.get(state_name)["y_pred"]

    metrics = {
        "State": state_name,
        "Explained Variance": explained_variance_score(y_true, y_pred),
        "Mean Absolute Error": mean_absolute_error(y_true, y_pred),
        "Mean Squared Error": mean_squared_error(y_true, y_pred),
        "R^2 Score": r2_score(y_true, y_pred),
        "Residual Sum of Errors": residual_sum_of_errors(y_true, y_pred),
        "Root Mean Squared Error": root_mean_squared_error(y_true, y_pred),
    }

    return metrics

In [ ]:
state_metrics_list = []

for state_name in states_of_interest:
    metrics = evaluate(state_name)
    state_metrics_list.append(metrics)

In [ ]:
# Convert the list of dictionaries to a DataFrame
metrics_df = pd.DataFrame(state_metrics_list)

In [ ]:
metrics_df

In [ ]:
metrics_df.shape

In [ ]:
print(metrics_df.to_latex(index=False, float_format="{:.1f}".format))

# Feature Importance
